# Importing Library

In [3]:
import pandas as pd
import numpy as np
import requests
import folium
import config
import logging
import requests
import datetime
import glob
import geopandas
from pyproj import Proj, transform

### Visualization

In [7]:
#EXAMPLE OF A HEATMAP
from folium.plugins import HeatMap
import folium

### Create a Map instance
mp = folium.Map(location=[52.5, 13.5], tiles = 'stamentoner', zoom_start=9, control_scale=True)

### Add heatmap to map instance
#### Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(bike_df).add_to(mp)

### Alternative syntax:
###m.add_child(HeatMap(points_array, radius=15))

#### Show map
mp

In [8]:
#Plotting

#accidents = test_accidents

m = folium.Map(location=[52.5, 13.5], tiles='Stamen Toner', zoom_start=15)
for i in range(len(bike_df)): # 11 accidents
    folium.CircleMarker(bike_df[i], radius=10, color='blue').add_to(m)

#    folium.Marker(each).add_to(m) # Very Heavy!!
m

# NextBike accumulated live data

In [29]:
# get data file names
path ='data/nextbike_livedata'
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
#print(dfs)
# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)

In [30]:
df.rename(columns = {"0":'Bike_ID', "1":'Datetime', "2":'Latitude', "3":'Longitude'}, inplace = True)

In [31]:
#To datetime
df.Datetime = pd.to_datetime(df.Datetime)
df.Datetime = df.Datetime.dt.strftime('%Y-%m-%d %H:%M')
df.Datetime = pd.to_datetime(df.Datetime, format='%Y-%m-%d %H:%M')
df.head()

,Bike_ID,Datetime,Latitude,Longitude
0,13013,2022-02-23 22:33:00,52.501855,13.316879
1,14632,2022-02-23 22:33:00,52.542932,13.419353
2,13396,2022-02-23 22:33:00,52.522604,13.400895
3,13391,2022-02-23 22:33:00,52.528143,13.392144
4,14553,2022-02-23 22:33:00,52.511937,13.408501


In [32]:
#DataFrame to list for mapping
#bike_df = list(df[[df.Latitude, df.Longitude]])
#ype(bike_df)

## Geometric System Conversion ETRS89 --> WGS84

In [38]:
#From WGS84 (lat/lon) --> UTM coordinate
p = Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=False)
x1, y1 = p(df[1:2].Longitude, df[1:2].Latitude) 
x1, y1

(array([392808.90647742]), array([5822601.28000405]))

In [34]:
#From WGS84 (lat/lon) --> UTM coordinate
p = Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=False)
df.xy = df.apply(lambda x: p(x.Longitude, x.Latitude))
df

AttributeError: 'Series' object has no attribute 'Longitude'

In [18]:
p = Proj(proj='utm', zone=33, ellps='WGS84', preserve_units=False)
x,y = p(-120.108, 34.36116666)
x, y

(-3753969.677835038, 15120462.670135964)

# LOR data

In [85]:
#Run this code when we use the notebook direcly on GoogleDrive
path_to_data_bzr = "data/LOR_shpfiles/lor_bzr.shp"
path_to_data_pgr = "data/LOR_shpfiles/lor_pgr.shp"
path_to_daza_plr = "data/LOR_shpfiles/lor_plr.shp"

gdf_bzr = geopandas.read_file(path_to_data_bzr)
gdf_pgr = geopandas.read_file(path_to_data_pgr)
gdf_plr = geopandas.read_file(path_to_daza_plr)

gdf_bzr = gdf_bzr.set_index("BZR_NAME")

In [87]:
gdf_plr.head()

,PLR_ID,PLR_NAME,geometry
0,08401245,Frauenviertel,"POLYGON ((398530.482 5807357.873, 398569.146 5..."
1,08401244,Waßmannsdorfer Chaussee,"POLYGON ((397994.196 5806511.717, 397993.161 5..."
2,08401246,Waltersdorfer Chaussee Ost,"POLYGON ((398909.474 5806709.670, 398907.398 5..."
3,08401243,Alt-Rudow,"POLYGON ((399260.777 5809071.777, 399251.106 5..."
4,08401242,Zittauer Straße,"POLYGON ((397378.822 5808471.734, 397377.582 5..."


In [94]:
str(gdf_bzr.geometry[0])

'POLYGON ((396597.49 5811370.953, 396601.64 5811367.834, 396605.554 5811364.89, 396623.22 5811351.586, 396627.838 5811347.309, 396793.534 5811209.042, 396795.729 5811207.211, 396825.947 5811181.999, 396826.535 5811181.368, 396846.684 5811164.557, 396865.769 5811148.615, 396886.577 5811131.252, 396906.578 5811114.563, 396926.451 5811097.967, 396946.343 5811081.37, 396948.371 5811079.682, 396961.492 5811068.764, 396966.258 5811064.904, 396969.747 5811062.228, 396971.048 5811061.223, 396974.791 5811059.312, 396979.204 5811057.388, 396986.708 5811054.476, 396994.993 5811051.109, 396995.4 5811050.941, 397002.341 5811047.309, 397007.062 5811043.71, 397075.536 5810986.934, 397163.068 5810914.358, 397285.468 5810812.871, 397291.342 5810807.86, 397293.169 5810806.125, 397296.988 5810802.433, 397297.544 5810801.673, 397300.536 5810797.576, 397306.911 5810786.286, 397308.116 5810783.964, 397310.839 5810779.922, 397312.502 5810777.981, 397317.104 5810773.904, 397330.647 5810762.557, 397331.809 581

In [81]:
#Plotting

bike_df = list(df[[df.Latitude, df.Longitude]])

m = folium.Map(location=[52.5, 13.5], tiles='Stamen Toner', zoom_start=15)
for i in range(len(bike_df)):
    folium.CircleMarker(bike_df[i], radius=10, color='blue').add_to(m)
m

KeyError: "None of [Index([(52.501855, 52.542932, 52.522604, 52.528143, 52.511937, 52.509865, 52.526701, 52.522394, 52.521414, 52.519586, 52.531871, 52.529268, 52.527079, 52.528398, 52.491439, 52.527236, 52.506974, 52.51589, 52.537646, 52.547469, 52.50655, 52.529268, 52.501996, 52.546722, 52.522172, 52.529278, 52.526796, 52.501991, 52.50681, 52.520178, 52.52969, 52.546777, 52.50442, 52.529804, 52.504466, 52.515824, 52.513688, 52.497789, 52.515779, 52.515824, 52.508691, 52.477018, 52.501951, 52.499691, 52.522291, 52.513001, 52.521182, 52.510099, 52.505689, 52.50502, 52.503751, 52.502507, 52.508243, 52.480093, 52.479984, 52.503456, 52.967277, 52.49468, 52.496439, 52.49411, 52.52077, 52.51297, 52.496825, 52.518621, 52.520231, 52.480221, 52.507978, 52.495323, 52.493948, 52.490469, 52.515809, 52.480041, 52.523451, 52.512848, 52.516334, 52.495925, 52.515237, 52.531169, 52.494576, 52.474145, 52.508019, 52.503279, 52.506428, 52.505691, 52.497434, 52.504341, 52.51576, 52.515753, 52.502779, 52.5